In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

Создадим датасет с данными скаковых лошадей.

Описание признаков:
- age: возраст лошади (год)
- height: высота лошади в холке (см)
- length: 'косая длина' лошади (см)
- weight: вес лошади (кг)
- chest: обхват груди (см)
- cannon: обхват пясти (см)
- breed: оценка породистости (балл)
- hay: количество рекомендованного суточного потребления сена (кг)
- price: цена лошади (руб)

In [2]:
# Создаём сэмпл
n_samples = 500
age = np.random.choice(9, n_samples) + 2
height = np.random.choice(21, n_samples) + 150
length = np.random.choice(26, n_samples) + 150
weight = np.random.choice(71, n_samples) + 490
chest = np.random.choice(21, n_samples) + 170
cannon = (np.random.choice(21, n_samples) + 190) / 10
breed = np.random.choice(10, n_samples) + 1
hay = np.round(weight * length / 6500, decimals=1)
# Вычисляем цену по формуле
price = 10000 * (length / height) + 2000 * (weight / height) + 10000 * (height / cannon) + 80000 * (breed / age)

In [3]:
# Создаём датафрейм
data = pd.DataFrame({'age': age, 'height': height, 'length': length, 'weight': weight, 'chest': chest,
                     'cannon': cannon, 'breed': breed, 'hay': hay, 'price': price})
data.head(10)

,age,height,length,weight,chest,cannon,breed,hay,price
0,3,158,168,526,170,19.7,5,13.6,230827.518259
1,8,157,165,551,185,20.2,6,14.0,155251.434698
2,10,168,152,543,189,19.5,8,12.7,165665.750916
3,4,168,168,506,189,20.4,4,13.1,178376.750700
4,8,161,172,529,177,20.6,10,14.0,195409.998191
5,7,152,170,560,171,20.7,7,14.6,171982.583270
6,8,152,150,541,182,19.4,5,12.5,145337.357569
7,3,151,150,539,181,20.5,5,12.4,224064.717601
8,2,152,175,501,172,20.2,3,13.5,213352.787910
9,4,150,167,555,181,19.9,4,14.3,173910.217755


In [4]:
# Посмотрим на признаки 5 самых дорогих и 5 самых дешёвых скакунов
data.sort_values(by='price', ascending=False)

,age,height,length,weight,chest,cannon,breed,hay,price
461,2,169,160,501,171,19.0,10,12.3,504343.818125
477,2,169,159,521,182,19.8,10,12.7,500927.499851
25,2,159,167,498,186,19.4,10,12.8,498726.058484
449,2,164,150,516,178,20.6,10,11.9,495050.674876
422,2,161,172,548,186,20.9,10,14.5,494524.176053
...,...,...,...,...,...,...,...,...,...
397,10,162,175,500,186,20.6,1,13.5,103616.085341
287,8,151,157,541,176,20.2,1,13.1,102315.389155
75,8,155,152,544,172,20.6,1,12.7,102068.524898
326,9,153,170,517,181,20.6,1,13.5,101030.014595


In [5]:
# Создадим модель линейной регрессии по нашим признакам, с целевой переменной - цена
X = data[['age', 'height', 'length', 'weight', 'chest', 'cannon', 'breed', 'hay']]
y = data['price']
# Выведем веса
reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')
# Выведем ошибку модели
pred_values = reg.predict(data[['age', 'height', 'length', 'weight', 'chest', 'cannon', 'breed', 'hay']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [-18503.43579118    614.02340548  -1367.23133246   -477.81880005
    -45.75965734  -4239.15506466  17678.65890252  14682.50430616]
Bias: 477535.08342745376
Error: 25994.04207914637


Ошибка, равная 25994.04, существенная, около 25% от наименьшей цены.

In [6]:
# Из обрабатываемых переменных, уберём признак 'hay', который не участвует в образовании цены, и посмотрим на ошибку модели
X = data[['age', 'height', 'length', 'weight', 'chest', 'cannon', 'breed']]
y = data['price']

reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['age', 'height', 'length', 'weight', 'chest', 'cannon', 'breed']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [-18502.43229999    609.35734236   -180.01050601   -110.03453863
    -53.24096516  -4214.08911133  17690.70097774]
Bias: 285741.1389781196
Error: 26010.735636438294


Ошибка снова слишком большая.

Введём новые переменные в наш датасет.
Для характеристики типа телосложения лошади используются индексы соотношений различных промеров.
- body_index: индекс формата лошади $\left(\text{косая длина}\over\text{высота в холке}\right)$
- density_index: индекс плотности $\left(\text{вес}\over\text{высота в холке}\right)$
- bone_index_inverse: $\left({1}\over\text{индекс костности}\right)$, где $\text{индекс костности}={\text{обхват пясти}\over\text{высота в холке}}$
- potential_index: $\left(\text{оценка породистости}\over\text{возраст}\right)$

In [7]:
data['body_index'] = data.length / data.height
data['density_index'] = data.weight / data.height
data['bone_index_inverse'] = data.height / data.cannon
data['potential_index'] = data.breed / data.age
data.head(5)

,age,height,length,weight,chest,cannon,breed,hay,price,body_index,density_index,bone_index_inverse,potential_index
0,3,158,168,526,170,19.7,5,13.6,230827.518259,1.063291,3.329114,8.020305,1.666667
1,8,157,165,551,185,20.2,6,14.0,155251.434698,1.050955,3.509554,7.772277,0.750000
2,10,168,152,543,189,19.5,8,12.7,165665.750916,0.904762,3.232143,8.615385,0.800000
3,4,168,168,506,189,20.4,4,13.1,178376.750700,1.000000,3.011905,8.235294,1.000000
4,8,161,172,529,177,20.6,10,14.0,195409.998191,1.068323,3.285714,7.815534,1.250000


In [8]:
# Построим модель на новых переменных, и посмотрим на ошибку модели
X = data[['body_index', 'density_index', 'bone_index_inverse', 'potential_index']]
y = data['price']

reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['body_index', 'density_index', 'bone_index_inverse', 'potential_index']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [10000.  2000. 10000. 80000.]
Bias: -9.604264050722122e-10
Error: 3.568129613995552e-11


Модель точно вычислила веса, ошибка стала очень маленькой:)